In [ ]:
import os
import cv2
import time
import math
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import mode

In [ ]:
%matplotlib inline
%config InlineBackend.rc={'figure.figsize': (12, 8)}

In [ ]:
name = '14080314_08_2019_08_10_34.mp4'
path = '/media/marcos/Dados/Projects/Results/PupilDeep/Frames/exams/avaliable/work'
exam = cv2.VideoCapture('{}/{}'.format(path, name))

path_original = '/media/marcos/Dados/Projects/Datasets/Exams'
exam_original = cv2.VideoCapture('{}/{}'.format(path_original, name))

fps = exam.get(cv2.CAP_PROP_FPS)
frame_count = int(exam.get(cv2.CAP_PROP_FRAME_COUNT))
duration = frame_count/fps

print(fps, frame_count, duration)

In [ ]:
frames = []
while True:
    _, frame = exam.read()
    
    if frame is None:
        break
        
    frames.append(frame)

print(len(frames))

In [ ]:
original_frames = []
while True:
    _, frame = exam_original.read()
    
    if frame is None:
        break
        
    original_frames.append(frame)

print(len(original_frames))

In [ ]:
path_information = '/media/marcos/Dados/Projects/Datasets/Exams/informations_exams'
file_information = '{}/{}.log'.format(path_information, name.replace('.mp4', ''))
print(file_information)

In [ ]:
patient = ''
param = ''
stimulus = []

with open(file_information, 'r') as information:
    ativate = False
    for line in information.readlines():            
        if 'PAC' in line:
            patient = line[line.rfind(':')+2:len(line)-1]
        if 'PRM' in line:
            param = line[line.rfind(':')+2:len(line)-1]
        if 'ATV' in line:
            stimul = line[line.rfind(':')+2:len(line)-1]
            init_stimul = line[0:line.rfind('-')-1]
            ativate = not ativate
        if 'DTV' in line:
            if ativate:                
                ativate = not ativate
                end_stimul = line[0:line.rfind('-')-1]
                stimulus.append({'stimul': stimul,
                                 'init_stimul': init_stimul,
                                 'end_stimul': end_stimul})

print(patient, param, stimulus, len(stimulus))

In [ ]:
def calc_frame_stimul(time_stimul, fps_movie):
    stimul = time_stimul
    iMinuts = int(stimul[0:stimul.find(':')])
    iSeconds = int(stimul[stimul.find(':')+1:stimul.rfind(':')])
    iMilliseconds = int(stimul[stimul.rfind(':')+1:len(stimul)])
    frame = math.ceil((iMinuts*(fps_movie*60))+(iSeconds*fps_movie)+((iMilliseconds*fps_movie)/100))
    return frame

In [ ]:
stimulus_treat = []
for stimul in stimulus:    
    aux = {'stimul': stimul['stimul'],
           'init_stimul': calc_frame_stimul(stimul['init_stimul'], fps),
           'end_stimul': calc_frame_stimul(stimul['end_stimul'], fps)}
    stimulus_treat.append(aux)

print(stimulus_treat)

In [ ]:
path_dataset = '/media/marcos/Dados/Projects/Results/PupilDeep/Labels'
file_dataset = '{}/{}_label.csv'.format(path_dataset, name.replace('.mp4', ''))
print(file_dataset)

In [ ]:
dataset = pd.read_csv(file_dataset)
print(dataset.head())

In [ ]:
alter_to = 'mode'
len_list_calc_normal = 40
len_list_calc_focus = 15
jump_after_stimulus = 50

aux = 0
for stimul in stimulus_treat:
    flash_dataset = dataset.loc[stimul['init_stimul']:stimul['end_stimul']]
    work_dataset = dataset.loc[aux:stimul['init_stimul']-1]

    frames_mod = mode(work_dataset['radius'])[0][0]
    frames_median = work_dataset['radius'].median()
    new_radius = frames_mod if alter_to == 'mode' else frames_median

    for index, item in flash_dataset.iterrows():
        dataset['radius'][index] = new_radius

    for index, item in work_dataset.iterrows():
        len_list = len_list_calc_normal
        if index > stimulus_treat[0]['init_stimul']:
            if index <= aux + jump_after_stimulus:
                len_list = len_list_calc_focus

        range_dataset = work_dataset.loc[index:index+len_list]
        frames_mod = mode(range_dataset['radius'])[0][0]
        frames_median = range_dataset['radius'].median()
        new_radius = frames_mod if alter_to == 'mode' else frames_median
        dataset['radius'][index] = new_radius

    aux = stimul['end_stimul']+1

mode_dataset = mode(dataset['radius'])[0][0]
std_dataset = dataset['radius'].std()
mean_dataset = dataset['radius'].mean()
print(mode_dataset, std_dataset, mean_dataset)

In [ ]:
dataset = pd.read_csv(file_dataset)
print(dataset.head())

In [ ]:
init_range, end_range = [], []
for stimul in stimulus_treat:
    init_range.append(stimul['init_stimul'])
    end_range.append(stimul['end_stimul'])

print(init_range, end_range)

In [ ]:
alter_to = 'mode'
len_list_calc_normal = 40
len_list_calc_focus = 15
jump_after_stimulus = 50

aux = 0
for i in range(0, len(init_range)):        
    flash_dataset = dataset.loc[init_range[i]:end_range[i]]
    work_dataset = dataset.loc[aux:init_range[i]-1]

    frames_mod = mode(work_dataset['radius'])[0][0]
    frames_median = work_dataset['radius'].median()
    new_radius = frames_mod if alter_to == 'mode' else frames_median

    for index, item in flash_dataset.iterrows():
        dataset['radius'][index] = new_radius

    for index, item in work_dataset.iterrows():
        len_list = len_list_calc_normal
        if index > init_range[0]:
            if index <= aux + jump_after_stimulus:
                len_list = len_list_calc_focus

        range_dataset = work_dataset.loc[index:index+len_list]
        frames_mod = mode(range_dataset['radius'])[0][0]
        frames_median = range_dataset['radius'].median()
        new_radius = frames_mod if alter_to == 'mode' else frames_median
        dataset['radius'][index] = new_radius

    aux = end_range[i]+1

mode_dataset = mode(dataset['radius'])[0][0]
std_dataset = dataset['radius'].std()
mean_dataset = dataset['radius'].mean()
print(mode_dataset, std_dataset, mean_dataset)

In [ ]:
plt.imshow(frames[1188])

In [ ]:
plt.imshow(original_frames[1188])